In [ ]:
!pip install ultralytics -q

In [ ]:
from ultralytics import YOLO
import numpy as np
import os, random, shutil
from pathlib import Path
from PIL import Image
from IPython.display import Image as DispImage, display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import yaml
import torch

In [ ]:
!unzip /content/drive/MyDrive/Processed_KITTI.zip -d /content/extracted_rgb

Streaming output truncated to the last 5000 lines.
  inflating: /content/extracted_rgb/train/labels/000663.txt  
  inflating: /content/extracted_rgb/train/labels/001328.txt  
  inflating: /content/extracted_rgb/train/labels/003267.txt  
  inflating: /content/extracted_rgb/train/labels/003549.txt  
  inflating: /content/extracted_rgb/train/labels/000095.txt  
  inflating: /content/extracted_rgb/train/labels/002505.txt  
  inflating: /content/extracted_rgb/train/labels/007384.txt  
  inflating: /content/extracted_rgb/train/labels/001879.txt  
  inflating: /content/extracted_rgb/train/labels/004520.txt  
  inflating: /content/extracted_rgb/train/labels/007162.txt  
  inflating: /content/extracted_rgb/train/labels/003762.txt  
  inflating: /content/extracted_rgb/train/labels/007170.txt  
  inflating: /content/extracted_rgb/train/labels/000175.txt  
  inflating: /content/extracted_rgb/train/labels/004143.txt  
  inflating: /content/extracted_rgb/train/labels/006723.txt  
  inflating: /conte

In [ ]:
!unzip /content/drive/MyDrive/Data_RGBD_v2_1.zip -d /content/extracted

Streaming output truncated to the last 5000 lines.
  inflating: /content/extracted/train/fused_images_train_v2/003304.png  
  inflating: /content/extracted/train/fused_images_train_v2/003305.png  
  inflating: /content/extracted/train/fused_images_train_v2/003306.png  
  inflating: /content/extracted/train/fused_images_train_v2/003308.png  
  inflating: /content/extracted/train/fused_images_train_v2/003309.png  
  inflating: /content/extracted/train/fused_images_train_v2/003310.png  
  inflating: /content/extracted/train/fused_images_train_v2/003311.png  
  inflating: /content/extracted/train/fused_images_train_v2/003312.png  
  inflating: /content/extracted/train/fused_images_train_v2/003313.png  
  inflating: /content/extracted/train/fused_images_train_v2/003314.png  
  inflating: /content/extracted/train/fused_images_train_v2/003315.png  
  inflating: /content/extracted/train/fused_images_train_v2/003316.png  
  inflating: /content/extracted/train/fused_images_train_v2/003317.png  


In [ ]:
!mv /content/extracted/train/fused_images_train_v2 /content/extracted/train/images


In [ ]:
!mv /content/extracted/val/fused_images_val_v2 /content/extracted/val/images


In [ ]:
!mv /content/extracted_rgb/val/rgb /content/extracted_rgb/val/images


In [ ]:

data_yaml = {
    'train': '/content/extracted/train/images',
    'val': '/content/extracted_rgb/val/images',
    'nc': 3,
    'names': ['Car', 'Pedestrian', 'Cyclist']
}

with open('/content/extracted/kitti_rgbd.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print('✅ YAML created at /content/extracted/kitti_rgbd.yaml')


✅ YAML created at /content/extracted/kitti_rgbd.yaml


In [ ]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 77.9MB/s]


In [ ]:
import json
from ultralytics import YOLO

# ==============================
# THÔNG SỐ CHUNG
# ==============================
data_yaml = '/content/extracted/kitti_rgbd.yaml'
epochs = 50
patience = 3
mixup = 0.1
device = 0
project = 'yolov8n-RGBD'
drive_base = '/content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/Log_Train/RGBD_Fused_Train/'

# ==============================
# DANH SÁCH SEED
# ==============================
seeds = [42, 123]


In [ ]:
def set_seed(seed):
    print(f"✅ Setting seed: {seed}")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


In [ ]:
for seed in seeds:
    set_seed(seed)

    # Load model
    model = YOLO('yolov8n.pt')  # đổi path nếu dùng model khác

    # Train
    results = model.train(
        data=data_yaml,
        imgsz=640,
        epochs=epochs,
        patience=patience,
        mixup=mixup,
        project=project,
        name=f'seed_{seed}',
        device=device
    )

    print(f"✅ Done seed {seed}. Weights saved in Ultralytics default folder under project/{project}/seed_{seed}")


✅ Setting seed: 42
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/extracted/kitti_rgbd.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=seed_42, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=3, perspective=0.0, plots=True,

100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    751897  ultralytics.nn.modules.head.Detect           [3, [64, 128, 256]]           
Model summary: 129 layers, 3,011,433 parameters, 3,011,417 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 67.3MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      37/50      3.04G     0.8987     0.5645     0.9385         96        640: 100%|██████████| 421/421 [04:02<00:00,  1.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:10<00:00,  2.37it/s]


                   all        749       3415      0.856      0.761      0.846      0.565
EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 34, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

37 epochs completed in 2.584 hours.
Optimizer stripped from yolov8n-RGBD/seed_123/weights/last.pt, 6.2MB
Optimizer stripped from yolov8n-RGBD/seed_123/weights/best.pt, 6.2MB

Validating yolov8n-RGBD/seed_123/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:10<00:00,  2.22it/s]


                   all        749       3415      0.883      0.749      0.842       0.57
                   Car        660       2763      0.913      0.889      0.954      0.746
            Pedestrian        186        476      0.883      0.589      0.746      0.416
               Cyclist        123        176      0.854      0.767      0.825      0.547
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to yolov8n-RGBD/seed_123
✅ Done seed 123. Weights saved in Ultralytics default folder under project/yolov8n-RGBD/seed_123


In [ ]:
!cp -r /content/yolov8n-RGBD "/content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/Log_Train/"


In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.7 MB/s eta 0:00:00


In [7]:
from ultralytics import YOLO
import numpy as np
import os, random, shutil
from pathlib import Path
from PIL import Image
from IPython.display import Image as DispImage, display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import yaml
import torch

In [8]:
# Cấu hình đường dẫn
model = YOLO('/content/drive/MyDrive/Khóa Luận/Data_Object/Code/Early_Fusion_RGBD_Yolov8/Log_Train/yolov8n-RGBD/seed_42/weights/best.pt')
test_dir = Path('/content/drive/MyDrive/Khóa Luận/Data_Object/testing/image_2')
out_dir  = Path('/content/drive/MyDrive/Khóa Luận/kitti_submit_yolov8_rgbd')
out_dir.mkdir(parents=True, exist_ok=True)

In [9]:
# Chạy predict từng ảnh, ghi ra KITTI-format .txt
cls_map = {0:'Car', 1:'Pedestrian', 2:'Cyclist'}

for img_path in sorted(test_dir.glob('*.*')):
    # predict 1 ảnh
    res = model.predict(source=str(img_path), save=False, verbose=False)[0]
    boxes  = res.boxes.xyxy.cpu().numpy()   # shape (N,4)
    classes= res.boxes.cls.cpu().numpy().astype(int)  # shape (N,)
    confs  = res.boxes.conf.cpu().numpy()  # shape (N,)

    # mở file output
    out_txt = out_dir / f"{img_path.stem}.txt"
    with open(out_txt, 'w') as f:
        for (x1,y1,x2,y2), c, p in zip(boxes, classes, confs):
            # KITTI 2D detection format:
            # <class> <trunc> <occl> <alpha> <x1> <y1> <x2> <y2>
            # <dim_x> <dim_y> <dim_z> <loc_x> <loc_y> <loc_z> <rot_y> <score>
            line = (
                f"{cls_map[c]} 0 0 0 "
                f"{x1:.2f} {y1:.2f} {x2:.2f} {y2:.2f} "
                "0 0 0 0 0 0 0 "
                f"{p:.4f}"
            )
            f.write(line + "\n")

print(f"Đã ghi xong {len(list(out_dir.glob('*.txt')))} files vào {out_dir}")

Đã ghi xong 7518 files vào /content/drive/MyDrive/Khóa Luận/kitti_submit_yolov8_rgbd


In [10]:
import zipfile
import os

# Thư mục cần nén
folder_to_zip = out_dir
zip_output = '/content/drive/MyDrive/kitti_submit_yolov8_rgbd.zip'

# Tạo file ZIP
with zipfile.ZipFile(zip_output, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            file_path = os.path.join(root, file)
            # Thêm file vào ZIP, giữ cấu trúc thư mục
            zipf.write(file_path, os.path.relpath(file_path, folder_to_zip))

print(f"Đã nén vào {zip_output}")

Đã nén vào /content/drive/MyDrive/kitti_submit_yolov8_rgbd.zip
